In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import text
from jproperties import Properties
from datetime import timedelta
from app.wallet_processing import Wallet
from app.wallet_processing import Cashflow

### This notebook will call the classes and functions for processing, acessing the data, load transactions etc.


In [2]:
w = Wallet(wallet_id = 1)
cashflow = Cashflow(wallet_id= 1)

In [3]:
w.get_last_position()

,date,id,brokerage_firm_id,asset,quantity,pu,value
0,2025-06-06,1,2,RECV3,300.00000000,14.19000000,4257.00000000
1,2025-06-06,1,1,POSI3,35.00000000,4.72000000,165.20000000
2,2025-06-06,1,2,BBAS3,150.00000000,21.93000000,3289.50000000


In [4]:
w.get_position('20250530')

,date,id,brokerage_firm_id,asset,quantity,pu,value
0,2025-05-30,1,2,RECV3,300.00000000,14.19000000,4257.00000000
1,2025-05-30,1,1,POSI3,35.00000000,4.72000000,165.20000000


In [5]:
#w.update_position() 

# this will update if there is any new transaction available in the database that weren't processed before, 
#based on the date (only works for new opeartions that are newer than the last date with position)

In [6]:
w.get_operations('20250530') # retrieve the ops that are > than the passed date;

,date_op,id,movement,market,date_expiration,brokerage_firm_id,asset,quantity,pu,value
0,2025-06-06,1,Compra,Mercado à Vista,2099-12-31,2,BBAS3,100,21.75,2175.00
1,2025-06-05,1,Compra,Mercado Fracionário,2099-12-31,2,BBAS3,50,22.29,1114.50


### CASHFLOW CLASS

In [7]:
cashflow.get_transaction_history(start_date = '20230101' ,end_date = '20250605')


,id,wallet_id,account,vd,bank_id,agency,date,value
0,1,1,1011,9,101,1,2025-05-20,4257.00
1,2,1,1011,9,101,1,2025-05-30,3454.70
2,3,1,1011,9,101,1,2025-05-31,-1000.00


In [8]:
# Example on how to insert a cashflow transfer
    
    #cashflow.money_transfer(1011, 9, 101, 1, '20250520', 4257.0)

total_coast = cashflow.get_transaction_history(start_date = '20010101' ,end_date = '20500101').value.sum()



### RESULTS 

In [9]:
import yfinance as yf


df = w.get_last_position()
df.head(3)

,date,id,brokerage_firm_id,asset,quantity,pu,value
0,2025-06-06,1,2,RECV3,300.00000000,14.19000000,4257.00000000
1,2025-06-06,1,1,POSI3,35.00000000,4.72000000,165.20000000
2,2025-06-06,1,2,BBAS3,150.00000000,21.93000000,3289.50000000


In [10]:
tickers = df.asset.values

tickers = " ".join(
    [ticker + '.SA' for ticker in tickers])

In [11]:
df_market_price = yf.Tickers(tickers).history('1mo')['Close']

###
df_market_price = df_market_price.reset_index().melt(id_vars = ['Date'])
df_market_price.columns = ['date', 'ticker', 'price_market']
df_market_price.ticker = [ticker.split('.')[0] for ticker in df_market_price.ticker]
df_market_price.loc[:, 'date'] = df_market_price.date.dt.date
df_market_price.set_index(['date', 'ticker'], inplace = True)

[*********************100%***********************]  3 of 3 completed


In [12]:
df_market_price.head(3)

,,price_market
date,ticker,
2025-05-27,BBAS3,24.570000
2025-05-28,BBAS3,24.080000
2025-05-29,BBAS3,23.700001


In [13]:
df.date = pd.to_datetime(df.date) # little trick
df_pnl = df.join(df_market_price, on = ['date', 'asset'])

In [ ]:
df_pnl['PnL_%'] = (df_pnl.price_market/df_pnl.pu.astype(float) -1 ) * 100
df_pnl['PnL_R$'] = df_pnl['PnL_%']/100 * df_pnl.value.astype(float)

In [15]:
df_pnl

,date,id,brokerage_firm_id,asset,quantity,pu,value,price_market,PnL(%),PnL(R$)
0,2025-06-06,1,2,RECV3,300.00000000,14.19000000,4257.00000000,14.500000,2.184637,93.000000
1,2025-06-06,1,1,POSI3,35.00000000,4.72000000,165.20000000,4.420000,-6.355931,-10.499997
2,2025-06-06,1,2,BBAS3,150.00000000,21.93000000,3289.50000000,21.709999,-1.003196,-33.000137
